faiss事前準備:

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒裝pandas:```pip install pandas```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [1]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入

In [2]:
input = "hey flora! Im having some issue with password! can you reset it for me?"

先準備faiss

讀embeddings資料

並把dataframe轉為numpy

In [3]:
import pandas as pd
import numpy as np

csv_file = 'data\Qembeddings.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

C:\Users\USER\AppData\Local\Temp\ipykernel_11248\926570023.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.022805,-0.003135,-0.013681,-0.036727,-0.044497,0.029007,-0.024501,-0.041485,-0.005907,-0.009333,...,-0.001190,-0.021287,0.009650,-0.010143,-0.028197,0.025374,-0.010055,-0.003249,-0.007062,-0.028197
1,-0.016840,-0.011369,-0.003460,-0.017197,-0.024066,0.012614,-0.030361,-0.040702,0.012908,-0.003005,...,0.013699,-0.041800,0.021296,-0.006463,-0.004542,-0.020491,-0.033731,0.010705,0.003454,-0.024437
2,-0.020366,-0.011115,0.016573,-0.021214,-0.017512,-0.002729,-0.011475,-0.030523,0.020134,-0.016496,...,0.023915,-0.022757,0.027797,-0.001901,-0.010009,0.003963,-0.034329,-0.025637,0.005378,-0.002792
3,-0.029621,-0.031300,-0.004523,-0.022191,-0.040698,0.019835,-0.013771,-0.023770,0.014309,-0.008721,...,0.015738,-0.005817,0.005927,0.000405,-0.026338,0.000488,-0.020086,-0.000731,0.001592,-0.020474
4,-0.014236,-0.011760,-0.014080,-0.002142,-0.036923,0.014444,-0.035155,-0.033881,0.009361,-0.001141,...,0.014509,-0.018397,0.017265,-0.009296,-0.004875,-0.010771,-0.011649,0.004901,0.003660,-0.035259


確認內容

In [4]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0228053741157054


寫入維度 並把data加入faiss的index

In [5]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

做問題的embedding

In [6]:
response = client.embeddings.create(
    input=input,
    model="text-embedding-ada-002"
)
question = np.array(response.data[0].embedding, ndmin=2)
print(question)

[[-0.00663677 -0.00735874 -0.03375349 ...  0.01932745 -0.00485504
  -0.02893156]]


TOP-k的設定並查找問題

In [7]:
k = 3

distances, indices = index.search(question, k)

拿答案

In [8]:
csv_question = 'data\manual.csv'
qdf = pd.read_csv(csv_question)
qarray = qdf.to_numpy()

看答案和距離

In [9]:
answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
print(answer)

[(array([' I forgot my password for my newly created Flora account thus I would like to reset my password.',
       'In the "Log In" section, there is an option "Forgot password?". By clicking, it would guide you through the process of resetting the password.Please try logging in with your old login method. If the problem still persists, we could manually migrate your old account to a new login method by the following process: 1) Sign up for a new account 2) Send us both your old and new account information, including login method, user name, and email account. If you use Apple Login without revealing your email, a screenshot of the Flora Setting page including the above mentioned information is needed. Thank you for your support and understanding! '],
      dtype=object), 0.20552632212638855), (array(["I already made an account for Flora and can't remember my password, but the app won't let me click 'forgot password'. Would you be able to send me a link so I can reset my password?",
 

# Prompt內容在此

In [10]:
history = ""
for i in range(0,k):
    history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
prompt = "Here is "+str(k)+" history questions and answers:\n"+history+"Here is my question:\n"+input
print(prompt)

Here is 3 history questions and answers:
1.  I forgot my password for my newly created Flora account thus I would like to reset my password.
In the "Log In" section, there is an option "Forgot password?". By clicking, it would guide you through the process of resetting the password.Please try logging in with your old login method. If the problem still persists, we could manually migrate your old account to a new login method by the following process: 1) Sign up for a new account 2) Send us both your old and new account information, including login method, user name, and email account. If you use Apple Login without revealing your email, a screenshot of the Flora Setting page including the above mentioned information is needed. Thank you for your support and understanding! 
2. I already made an account for Flora and can't remember my password, but the app won't let me click 'forgot password'. Would you be able to send me a link so I can reset my password?
We can help migrate your accoun

創assistant,thread,message!

In [11]:
from tabulate import tabulate

assistant = client.beta.assistants.create(
    name="Flora Customer Service",
    instructions="You are customer service from Flora.You can only answer questions related to Flora. when answering any question, you must always answer base on history's questions and answers which user have provided.",
    model="gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= prompt
)


沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [12]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

┌──────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ assistant    │ content                                                                                                                                                                                                   │
├──────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ asst_YEonNZVvA1LotrBgS1qznS1N                                                                                                                                                                             │
│ created_at   │ 1706172425                                                                                         

In [13]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_GGDmIBSg4WUbrKkxP43HEoTn │
│ created_at │ 1706172425                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [14]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

┌──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [16]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

┌─────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run             │ content                                                                                                                                                                                                   │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id              │ run_lB0PNMrd9roB9EZgOCjBw6Xt                                                                                                                                                                              │
│ assistant_id    │ asst_YEonNZVvA1LotrBgS1qznS1N                                                       

跑完後打開查看history messages!

In [18]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

最後一個訊息顯示

In [42]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data[0].content[0].text.value)

We could not manually set your password. However, you could follow the instructions by clicking "Forgot password?" in the "Log In" section to guide you through the process of resetting the password. Remember not to swipe Flora while you go to check the passcode.
